In [17]:
import numpy as np
from numpy.random import default_rng
import math
from math import ceil, log
from numpy import e
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator
from sklearn import preprocessing
from sympy.matrices import Matrix, eye, zeros, ones, diag, GramSchmidt

In [2]:
def psi1(t, c1, c2, c3, c4, epsilon, delta, chi):
    return c1*epsilon*e**(-t*squareroot(epsilon, delta, chi)/2)/(delta + 2*chi) + \
    c2*epsilon*t*e**(-t*squareroot(epsilon, delta, chi)/2)/(delta + 2*chi) + \
    c2*squareroot(epsilon, delta, chi)*e**(-t*squareroot(epsilon, delta, chi)/2)/(delta + 2*chi) + \
    c3*epsilon*e**(t*squareroot(epsilon, delta, chi)/2)/(delta + 2*chi) + \
    c4*epsilon*t*e**(t*squareroot(epsilon, delta, chi)/2)/(delta + 2*chi) + \
    c4*(delta**2 + 2*delta*chi + epsilon**2)*e**(t*squareroot(epsilon, delta, chi)/2) \
    /((delta + 2*chi)*squareroot(epsilon, delta, chi))
    
def psi2(t, c1, c2, c3, c4, epsilon, delta, chi):
    return -c1*squareroot(epsilon, delta, chi)*e**(-t*squareroot(epsilon, delta, chi)/2)/(delta + 2*chi) + \
    c2*epsilon*e**(-t*squareroot(epsilon, delta, chi)/2)/(delta + 2*chi) - \
    c2*t*squareroot(epsilon, delta, chi)*e**(-t*squareroot(epsilon, delta, chi)/2)/(delta + 2*chi) + \
    c3*squareroot(epsilon, delta, chi)*e**(t*squareroot(epsilon, delta, chi)/2)/(delta + 2*chi) + \
    c4*epsilon*e**(t*squareroot(epsilon, delta, chi)/2)/(delta + 2*chi) + \
    c4*t*squareroot(epsilon, delta, chi)*e**(t*squareroot(epsilon, delta, chi)/2)/(delta + 2*chi)
    
def phi1(t, c1, c2, c3, c4, epsilon, delta, chi):
    return c1*e**(-t*squareroot(epsilon, delta, chi)/2) + \
    c2*t*e**(-t*squareroot(epsilon, delta, chi)/2) + \
    c3*e**(t*squareroot(epsilon, delta, chi)/2) + \
    c4*t*e**(t*squareroot(epsilon, delta, chi)/2)

def phi2(t, c2, c4, epsilon, delta, chi):
    return c2*e**(-t*squareroot(epsilon, delta, chi)/2) + \
    c4*e**(t*squareroot(epsilon, delta, chi)/2)

def check_sqrt(epsilon, delta, chi):
    return -delta**2 - 2*delta*chi - epsilon**2 < 0

def squareroot(epsilon, delta, chi):
    result = -delta**2 - 2*delta*chi - epsilon**2
    return math.sqrt(result)

def func(xp):
    return ((abs(psi1(xp[0], xp[1], xp[2], xp[3], xp[4], xp[5], xp[6], xp[7])))**2 + \
            (abs(psi2(xp[0], xp[1], xp[2], xp[3], xp[4], xp[5], xp[6], xp[7])))**2 + \
            (abs(phi1(xp[0], xp[1], xp[2], xp[3], xp[4], xp[5], xp[6], xp[7])))**2 + \
            (abs(phi2(xp[0], xp[2], xp[4], xp[5], xp[6], xp[7])))**2)

In [3]:
def normalize_func(xp):
    psi_squared = (abs(psi1(xp[0], xp[1], xp[2], xp[3], xp[4], xp[5], xp[6], xp[7])))**2 + \
            (abs(psi2(xp[0], xp[1], xp[2], xp[3], xp[4], xp[5], xp[6], xp[7])))**2
    phi_squared = (abs(phi1(xp[0], xp[1], xp[2], xp[3], xp[4], xp[5], xp[6], xp[7])))**2 + \
            (abs(phi2(xp[0], xp[2], xp[4], xp[5], xp[6], xp[7])))**2
    return [psi_squared**2/(psi_squared**2+phi_squared**2), phi_squared**2/(psi_squared**2+phi_squared**2)]

In [12]:
def a(xp):
    psi1p = psi1(xp[0], xp[1], xp[2], xp[3], xp[4], xp[5], xp[6], xp[7])
    psi2p = psi2(xp[0], xp[1], xp[2], xp[3], xp[4], xp[5], xp[6], xp[7])
    psi_squared = (abs(psi1p))**2 + (abs(psi2p))**2
    second_half = psi2p*np.conj(psi1p) - psi1p*np.conj(psi2p)
    return complex(psi_squared, second_half)

def b(xp):
    psi1p = psi1(xp[0], xp[1], xp[2], xp[3], xp[4], xp[5], xp[6], xp[7])
    psi2p = psi2(xp[0], xp[1], xp[2], xp[3], xp[4], xp[5], xp[6], xp[7])
    phi1p = phi1(xp[0], xp[1], xp[2], xp[3], xp[4], xp[5], xp[6], xp[7])
    phi2p = phi2(xp[0], xp[2], xp[4], xp[5], xp[6], xp[7])
    first_half = psi1p*np.conj(phi1p) + psi2p*np.conj(phi2p)
    second_half = psi2p*np.conj(phi1p) - psi1p*np.conj(phi2p)
    return complex(first_half, second_half)

def c(xp):
    psi1p = psi1(xp[0], xp[1], xp[2], xp[3], xp[4], xp[5], xp[6], xp[7])
    psi2p = psi2(xp[0], xp[1], xp[2], xp[3], xp[4], xp[5], xp[6], xp[7])
    phi1p = phi1(xp[0], xp[1], xp[2], xp[3], xp[4], xp[5], xp[6], xp[7])
    phi2p = phi2(xp[0], xp[2], xp[4], xp[5], xp[6], xp[7])
    first_half = phi1p*np.conj(psi1p) + phi2p*np.conj(psi2p)
    second_half = phi2p*np.conj(psi1p) - phi1p*np.conj(psi2p)
    return complex(first_half, second_half)

def d(xp):
    phi1p = phi1(xp[0], xp[1], xp[2], xp[3], xp[4], xp[5], xp[6], xp[7])
    phi2p = phi2(xp[0], xp[2], xp[4], xp[5], xp[6], xp[7])
    phi_squared = (abs(phi1p))**2 + (abs(phi2p))**2
    second_half = phi2p*np.conj(phi1p) - phi1p*np.conj(phi2p)
    return complex(phi_squared, second_half)

In [15]:
x = [0, -8.042455772869037, 0.4104156269769683, 0.7626971134550099, 0.12854464458518508, 0.7365235537491663, 0.5104429911912247, -0.8622287451860423]

arr = np.array([[a(x), b(x)], [c(x), d(x)]])



In [16]:
np.linalg.eig(arr)

(array([5.02542450e-15+2.59996203e-15j, 7.77122319e+01-2.59996203e-15j]),
 array([[ 0.82805518+0.j        , -0.51194957+0.22854377j],
        [ 0.51194957+0.22854377j,  0.82805518+0.j        ]]))

In [37]:
p1 = complex(5.02542450*10**(-15), 2.59996203*10**(-15))
temp = complex(5.02542450*10**(-15), 2.59996203*10**(-15))
p2 = complex(7.77122319*10, -2.59996203*10**(-15))
p1 = p1/(temp + p2)
p2 = p2/(temp+p2)
print("p1 ", p1)
print("p2 ", p2)
p1*np.log(p1) + p2*np.log(p2)

p1  (6.466709779313391e-17+3.345627794277776e-17j)
p2  (1-3.345627794277776e-17j)


(-2.4189181165605785e-15-1.245773113810907e-15j)

In [21]:
from sympy import *
from sympy.abc import x
from sympy.printing.latex import LatexPrinter, print_latex
from sympy.functions import exp

In [22]:
t, chi, delta, epsilon, c1, c2, c3, c4 = symbols('t chi Delta epsilon c1 c2 c3 c4')
psi1, psi2 = symbols('psi_1, psi_2', cls=Function)
phi1, phi2 = symbols('phi_1, phi_2', cls=Function)

In [25]:
exprpsi1 = c1*epsilon*exp(-t*sqrt(-delta**2 - 2*delta*chi - epsilon**2)/2)/(delta + 2*chi) + \
    c2*epsilon*t*exp(-t*sqrt(-delta**2 - 2*delta*chi - epsilon**2)/2)/(delta + 2*chi) + \
    c2*sqrt(-delta**2 - 2*delta*chi - epsilon**2)*exp(-t*sqrt(-delta**2 - 2*delta*chi - epsilon**2)/2)/(delta + 2*chi) + \
    c3*epsilon*exp(t*sqrt(-delta**2 - 2*delta*chi - epsilon**2)/2)/(delta + 2*chi) + \
    c4*epsilon*t*exp(t*sqrt(-delta**2 - 2*delta*chi - epsilon**2)/2)/(delta + 2*chi) + \
    c4*(delta**2 + 2*delta*chi + epsilon**2)*exp(t*sqrt(-delta**2 - 2*delta*chi - epsilon**2)/2) \
    /((delta + 2*chi)*sqrt(-delta**2 - 2*delta*chi - epsilon**2))
    
exprpsi2 = -c1*sqrt(-delta**2 - 2*delta*chi - epsilon**2)*exp(-t*sqrt(-delta**2 - 2*delta*chi - epsilon**2)/2)/(delta + 2*chi) + \
    c2*epsilon*exp(-t*sqrt(-delta**2 - 2*delta*chi - epsilon**2)/2)/(delta + 2*chi) - \
    c2*t*sqrt(-delta**2 - 2*delta*chi - epsilon**2)*exp(-t*sqrt(-delta**2 - 2*delta*chi - epsilon**2)/2)/(delta + 2*chi) + \
    c3*sqrt(-delta**2 - 2*delta*chi - epsilon**2)*exp(t*sqrt(-delta**2 - 2*delta*chi - epsilon**2)/2)/(delta + 2*chi) + \
    c4*epsilon*exp(t*sqrt(-delta**2 - 2*delta*chi - epsilon**2)/2)/(delta + 2*chi) + \
    c4*t*sqrt(-delta**2 - 2*delta*chi - epsilon**2)*exp(t*sqrt(-delta**2 - 2*delta*chi - epsilon**2)/2)/(delta + 2*chi)
    
exprphi1 = c1*exp(-t*sqrt(-delta**2 - 2*delta*chi - epsilon**2)/2) + \
    c2*t*exp(-t*sqrt(-delta**2 - 2*delta*chi - epsilon**2)/2) + \
    c3*exp(t*sqrt(-delta**2 - 2*delta*chi - epsilon**2)/2) + \
    c4*t*exp(t*sqrt(-delta**2 - 2*delta*chi - epsilon**2)/2)

exprphi2 = c2*exp(-t*sqrt(-delta**2 - 2*delta*chi - epsilon**2)/2) + \
    c4*exp(t*sqrt(-delta**2 - 2*delta*chi - epsilon**2)/2)

In [29]:
exprA = Abs(exprpsi1)**2 + Abs(exprpsi2)**2 + I*(exprpsi2*conjugate(exprpsi1) - exprpsi1*conjugate(exprpsi2))
exprB = exprpsi1*conjugate(exprphi1) + exprpsi2*conjugate(exprphi2) + I*(exprpsi2*conjugate(exprphi1) - exprpsi1*conjugate(exprphi2))
exprC = exprphi1*conjugate(exprpsi1) + exprphi2*conjugate(exprpsi2) + I*(exprphi2*conjugate(exprpsi1) - exprphi1*conjugate(exprpsi2))
exprD = Abs(exprphi1)**2 + Abs(exprphi2)**2 + I*(exprphi2*conjugate(exprphi1) - exprphi1*conjugate(exprphi2))


In [30]:
m = Matrix([[exprA, exprB], [exprC, exprD]])

In [ ]:
p, d = m.diagonalize()

In [ ]:
m.eigenvals()